# Compliance Risk Management Dashboard

**Purpose:** Instant-access reports for overall compliance risk profile, regulatory breaches, top risks, and incident tracking.

**Audience:** Chief Compliance Officer, Chief Risk Officer, Board of Directors, Internal Audit, Regulatory Affairs

---

## The Board Emergency Scenario

**Thursday, 4:00 PM.** Board Chair calls: *"Emergency board meeting tomorrow 9 AM. We need:**
1. **"What is our overall compliance risk profile, and how has it changed?"**
2. **"Have we had any material regulatory breaches or supervisory findings?"**
3. **"What are the top 5 compliance risks that could lead to a fine or license restriction?"**
4. **"What's our remediation status for open issues?"**

**Traditional Response:** "I need 3 days to compile data from Risk, Legal, Audit, and Operations..." (Board panics. Crisis mode activated. Weekend ruined.)

**With This Notebook:** "I'll have the full board pack ready by 7 PM tonight." (Runs 4 queries, exports dashboards, pulls remediation tracker, delivers comprehensive presentation. Board confident. Weekend saved.)

---

## Daily Routine: CCO Morning Workflow

### 7:30 AM - 8:00 AM: Risk Dashboard Review
- Run Enterprise Compliance Risk Dashboard
- Check RAG status for all risk categories
- Flag any risks exceeding appetite
- Check Regulatory Breach Tracker
- Review Top 5 Risks Dashboard

### 9:00 AM - 12:00 PM: Risk Committee Meetings
- **Monday:** AML Committee
- **Tuesday:** Sanctions Committee
- **Wednesday:** Data Governance Committee
- **Thursday:** Operational Risk Committee
- **Friday:** Executive Risk Committee

### Afternoon: Remediation Management & Board Reporting
- Review remediation action tracker
- Follow up with action owners on overdue items
- Monthly: Prepare board risk pack (first Friday)

---

## 1. Environment Setup

**Business Context:** Establish connection to enterprise risk management platform.

**Data Sources:**
- `CRMA_AGG_DT_CUSTOMER_360` - Customer risk profiles
- `PAYA_AGG_DT_TRANSACTION_ANOMALIES` - Transaction monitoring alerts
- `SANC_SANCTIONS_DATA` - Sanctions exposure


In [ ]:
# Setup
from snowflake.snowpark.context import get_active_session
import streamlit as st
import pandas as pd
from datetime import datetime

session = get_active_session()
session.use_database('AAA_DEV_SYNTHETIC_BANK')
session.use_schema('CRM_AGG_001')
st.success('Connected to Risk Management Platform')

## 2. Question 1: Overall Compliance Risk Profile

**Board Question:** "What is our overall compliance risk profile?"

**Why This Matters:**
- **Board Governance:** Directors need to understand enterprise risk exposure
- **Strategic Planning:** Inform risk appetite and business strategy
- **Regulatory Oversight:** Demonstrate effective risk management to regulators

**What We're Measuring:**
- High-risk customer concentration
- PEP exposure
- Sanctions screening activity
- Transaction monitoring alerts
- Geographic risk concentration


In [ ]:
# Query: Overall Risk Profile (Using Shared View)
# This uses CRMA_AGG_VW_CUSTOMER_RISK_PROFILE - a single source of truth for risk metrics

df_raw = session.table('CRMA_AGG_VW_CUSTOMER_RISK_PROFILE').to_pandas()

# Map view columns to expected notebook columns for compatibility
df = pd.DataFrame({
    'TOTAL_CUSTOMERS': [df_raw['TOTAL_CUSTOMERS'].iloc[0]],
    'CRITICAL_RISK': [df_raw['CRITICAL_RISK_COUNT'].iloc[0]],
    'HIGH_RISK': [df_raw['HIGH_RISK_COUNT'].iloc[0]],
    'PEP_CUSTOMERS': [df_raw['PEP_COUNT'].iloc[0]],
    'SANCTIONS_HITS': [df_raw['SANCTIONS_COUNT'].iloc[0]],
    'CUSTOMERS_WITH_ANOMALIES': [df_raw['ANOMALY_COUNT'].iloc[0]],
    'HIGH_RISK_PERCENTAGE': [df_raw['CRITICAL_RISK_PCT'].iloc[0] + df_raw['HIGH_RISK_PCT'].iloc[0]]
})

st.subheader('Enterprise Compliance Risk Profile')
st.dataframe(df, use_container_width=True)

## 3. Risk Profile Dashboard

**Business Context:** Board-ready risk metrics for governance oversight.


In [ ]:
# Metrics
total = df['TOTAL_CUSTOMERS'].iloc[0]
critical = df['CRITICAL_RISK'].iloc[0]
high = df['HIGH_RISK'].iloc[0]
pep = df['PEP_CUSTOMERS'].iloc[0]
sanctions = df['SANCTIONS_HITS'].iloc[0]
anomalies = df['CUSTOMERS_WITH_ANOMALIES'].iloc[0]
high_pct = df['HIGH_RISK_PERCENTAGE'].iloc[0]

st.subheader('Risk Concentration Metrics')
col1, col2, col3 = st.columns(3)

with col1:
    st.metric('Total Customers', f'{total:,}')
    st.metric('Critical Risk', f'{critical:,}', help='Requires senior management oversight')
    st.metric('High Risk', f'{high:,}', help='Enhanced monitoring required')

with col2:
    st.metric('PEP Customers', f'{pep:,}', f'{pep/total*100:.2f}%')
    st.metric('Sanctions Hits', f'{sanctions:,}', help='Requires investigation')
    st.metric('Anomaly Flags', f'{anomalies:,}')

with col3:
    st.metric('High Risk %', f'{high_pct:.1f}%')
    if high_pct > 15:
        st.error('High risk concentration above threshold')
    elif high_pct > 10:
        st.warning('Monitor risk concentration')
    else:
        st.success('Risk profile acceptable')

# Risk Distribution
st.subheader('Risk Classification Distribution')
risk_df = pd.DataFrame({
    'Risk Level': ['Critical', 'High', 'Medium/Low'],
    'Count': [critical, high, total - critical - high]
})
st.bar_chart(risk_df.set_index('Risk Level'))

## 4. Export Risk Report

**Purpose:** Generate board-ready risk assessment documentation.


In [ ]:
# Export
ts = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'Compliance_Risk_Profile_{ts}.csv'

export_df = df.copy()
export_df['REPORT_DATE'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
export_df['PREPARED_BY'] = 'Chief Compliance Officer'
export_df['BOARD_PRESENTATION'] = 'Enterprise Risk Assessment'

csv_data = export_df.to_csv(index=False)

st.download_button(
    label='Download Risk Assessment (CSV)',
    data=csv_data,
    file_name=filename,
    mime='text/csv'
)

st.success(f'Risk assessment ready for board presentation: {filename}')

## Key Takeaways for CCO

**Always have these 3 reports ready:**
1. Enterprise Compliance Risk Dashboard
2. Regulatory Breach & Incident Tracker
3. Top 5 Risks Dashboard

**Response time targets:**
- Board inquiry: <1 hour for summary, <4 hours for full board pack
- Regulator inquiry: <24 hours for risk profile, <5 days for detailed analysis
- Incident escalation: Immediate notification for critical incidents

**Key risk metrics to monitor:**
- **Residual risk:** All categories within appetite
- **Remediation completion:** >80% of actions on track
- **Early warning indicators:** <80% of threshold = green, 80-100% = amber, >100% = red
- **Incident rate:** Declining trend over 12 months

**Escalation Protocols:**

### Level 1: Daily Escalation (Morning Email)
**Triggers:** Any risk exceeding appetite threshold, New critical incident discovered, Early warning indicator breach
**Action:** Email CEO + CRO with summary + recommended actions

### Level 2: Weekly Escalation (Friday Board Call)
**Triggers:** Multiple risks exceeding appetite, Material regulatory breach, Remediation significantly behind schedule
**Action:** Executive committee meeting with action plan

### Level 3: Board Escalation (Emergency Meeting)
**Triggers:** Regulatory enforcement action initiated, Material fine imposed, License restriction threatened, Systemic control failure identified
**Action:** Emergency board meeting with full analysis + remediation plan + external advisor engagement

**Proactive Monitoring:**
- Run risk dashboard daily (7:30 AM)
- Review incident tracker daily (morning)
- Analyze top risks weekly (Friday)
- Update risk register monthly (first week)
- Board reporting quarterly

---

**Last Updated:** December 2025  
**Owner:** Chief Compliance Officer  
**Reviewers:** Chief Risk Officer, Head of Internal Audit, Board Risk Committee